In [1]:
import warnings
warnings.filterwarnings('ignore')
import stwcs
import glob
import sys
import os
import shutil
import time
import filecmp
import astroquery
import progressbar
import copy
import requests
import random
import astropy.wcs as wcs
import numpy as np
from contextlib import contextmanager
from astropy import units as u
from astropy.utils.data import clear_download_cache,download_file
from astropy.io import fits
from astropy.table import Table, Column, unique
from astropy.time import Time
from astroscrappy import detect_cosmics
from stwcs import updatewcs
from scipy.interpolate import interp1d

# Internal dependencies
from common import Constants
from common import Options
from common import Settings
from common import Util
from nbutils import get_filter, get_instrument, get_filter, get_zpt, add_visit_info, organize_reduction_tables

@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

with suppress_stdout():
    from drizzlepac import tweakreg,astrodrizzle,catalogs,photeq
    from astroquery.mast import Observations
    from astropy.coordinates import SkyCoord

In [2]:
acceptable_filters = [
    'F220W','F250W','F330W','F344N','F435W','F475W','F550M','F555W',
    'F606W','F625W','F660N','F660N','F775W','F814W','F850LP','F892N',
    'F098M','F105W','F110W','F125W','F126N','F127M','F128N','F130N','F132N',
    'F139M','F140W','F153M','F160W','F164N','F167N','F200LP','F218W','F225W',
    'F275W','F280N','F300X','F336W','F343N','F350LP','F373N','F390M','F390W',
    'F395N','F410M','F438W','F467M','F469N','F475X','F487N','F547M',
    'F600LP','F621M','F625W','F631N','F645N','F656N','F657N','F658N','F665N',
    'F673N','F680N','F689M','F763M','F845M','F953N','F122M','F160BW','F185W',
    'F218W','F255W','F300W','F375N','F380W','F390N','F437N','F439W','F450W',
    'F569W','F588N','F622W','F631N','F673N','F675W','F702W','F785LP','F791W',
    'F953N','F1042M']

### HST

In [3]:
ra, dec = 31.4625, -58.49072
coord = SkyCoord(ra, dec, frame='icrs', unit='deg')
radius = 5*u.arcmin

In [4]:
obsTable = Observations.query_region(coord, radius=radius)

In [5]:
obsTable = obsTable.filled()

In [6]:
masks = []
masks.append([t.upper()=='HST' for t in obsTable['obs_collection']])
masks.append([p.upper()=='IMAGE' for p in obsTable['dataproduct_type']])
masks.append([any(l) for l in list(map(list,zip(*[[det in inst.upper()
            for inst in obsTable['instrument_name']]
            for det in ['ACS','WFC','WFPC2']])))])
# Added mask to remove calibration data from search
masks.append([f.upper()!='DETECTION' for f in obsTable['filters']])
masks.append([i.upper()!='CALIBRATION' for i in obsTable['intentType']])

In [7]:
mask = [all(l) for l in list(map(list, zip(*masks)))]
obsTable = obsTable[mask]

In [8]:
obsTable.sort('t_min')

In [14]:
obsTable_ = obsTable[5:15] #obsTable[obsTable['instrument_name'] == 'WFC3/UVIS'][0] #

In [15]:
obsTable_

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,distance
str11,str11,str17,str12,str4,str13,str8,str32,str60,str73,float64,float64,str10,str20,int64,float64,float64,float64,float64,float64,str122,float64,str23,str8,int64,str26502,str141,str142,str6,bool,float64,str9,float64
science,HST,HAP-SVM,WFC3/UVIS,HAP,F814W,Optical,SPT0205,N/A,hst_13677_05_wfc3_uvis_f814w_icn105,31.4428,-58.4852,image,"Perlmutter, Saul",3,56936.65771507,56936.66107153,290.0,702.5,961.2,N/A,56936.9166666,13677,HAP,999999,POLYGON 31.48024102338906 -58.49485001982922 31.477338208963765 -58.49830659952739 31.462920791766027 -58.51536838504191 31.404061239638263 -58.49942854119178 31.39393371581553 -58.496662776079354 31.393681070553434 -58.4965966484591 31.393407380175546 -58.496519506868154 31.39319685930626 -58.49645339413958 31.392923155794048 -58.49638725706965 31.392796834987422 -58.496354192329136 31.392565255173825 -58.4962880705403 31.392375782837615 -58.49623297008018 31.391996824344258 -58.49613377387377 31.391744186538613 -58.49606764244923 31.391365247418392 -58.495957438830956 31.391091567113765 -58.495880292312634 31.39092314415749 -58.4958362035661 31.390628405002076 -58.495759047728974 31.390333667142432 -58.49568189121651 31.390102096404064 -58.49561576499321 31.389660001809226 -58.4954945253869 31.389260014877 -58.495384307026626 31.389112657116904 -58.49534022451172 31.38896528317183 -58.4953071473787 31.388754777939653 -58.49524102738033 31.38860740474836 -58.495207949837294 31.38848109225121 -58.49517488085858 31.38801796531646 -58.4950426213249 31.387912697401813 -58.4950205661442 31.38776534286072 -58.49497648208581 31.387554824874794 -58.494921365674266 31.3869443350484 -58.4947560233572 31.3866075626473 -58.49463481813246 31.400653685716325 -58.477482415779036 31.403393794425046 -58.47415958539489 31.48024102338906 -58.49485001982922 POLYGON 31.498087162080626 -58.473635735987905 31.480535465985465 -58.49448674810123 31.403667774411325 -58.47382950025512 31.420476291835048 -58.45321998319676 31.498087162080626 -58.473635735987905,mast:HST/product/hst_13677_05_wfc3_uvis_f814w_icn105_drc.jpg,mast:HST/product/hst_13677_05_wfc3_uvis_f814w_icn105_drc.fits,PUBLIC,False,nan,26617203,0.0


In [16]:
productlist = None

#### All products for single observation

In [15]:
prlst = Observations.get_product_list(obs)

In [16]:
prlst

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str8,str3,str5,str35,str64,str1,str67,str9,str28,str11,str1,str7,str20,str5,str50,int64,str8,str6,int64,str9
24109735,HST,image,jbqz25gqq,DADS LOG file,S,mast:HST/product/jbqz25gqq_log.txt,INFO,--,LOG,--,CALACS,--,12477,jbqz25gqq_log.txt,14131,24835771,PUBLIC,1,F814W
24109735,HST,image,jbqz25gqq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gqq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gqq_flc.fits,168327360,24835771,PUBLIC,2,F814W
24109735,HST,image,jbqz25gqq,Preview-Full,S,mast:HST/product/jbqz25gqq_raw.jpg,PREVIEW,--,--,--,CALACS,--,12477,jbqz25gqq_raw.jpg,1238054,24835771,PUBLIC,1,F814W
24109735,HST,image,jbqz25gqq,DADS FLT file - Calibrated exposure ACS/WFC3/STIS/COS,S,mast:HST/product/jbqz25gqq_flt.fits,SCIENCE,--,FLT,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gqq_flt.fits,168442560,24835771,PUBLIC,2,F814W
24109735,HST,image,jbqz25gqq,DADS RAW file - Raw exposure COS/NICMOS/STIS/WFC3/ACS,S,mast:HST/product/jbqz25gqq_raw.fits,SCIENCE,--,RAW,--,CALACS,--,12477,jbqz25gqq_raw.fits,34344000,24835771,PUBLIC,1,F814W
24109735,HST,image,jbqz25gqq,Preview-Full,S,mast:HST/product/jbqz25gqq_flc.jpg,PREVIEW,--,--,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gqq_flc.jpg,1638316,24835771,PUBLIC,2,F814W
24109735,HST,image,jbqz25gqq,Preview-Full,S,mast:HST/product/jbqz25gqq_flt.jpg,PREVIEW,--,--,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gqq_flt.jpg,1623126,24835771,PUBLIC,2,F814W
24109735,HST,image,jbqz25gqq,DADS SPT file - Engineering telemetry ACS/WFC3/NICMOS/COS/STIS,S,mast:HST/product/jbqz25gqq_spt.fits,AUXILIARY,--,SPT,--,CALACS,--,12477,jbqz25gqq_spt.fits,57600,24835771,PUBLIC,1,F814W
24109735,HST,image,jbqz25gqq,DADS TRL file - Processing log,S,mast:HST/product/jbqz25gqq_trl.fits,AUXILIARY,--,TRL,--,CALACS,--,12477,jbqz25gqq_trl.fits,66240,24835771,PUBLIC,1,F814W


In [17]:
splt = [prod.split('_')[-1] for prod in prlst['productFilename']]

In [18]:
np.unique(np.array(splt))

array(['asn.fits', 'drc.fits', 'drc.jpg', 'drz.fits', 'drz.jpg',
       'flc.fits', 'flc.jpg', 'flt.fits', 'flt.jpg', 'hlet.fits',
       'jif.fits', 'jit.fits', 'log.txt', 'point-cat.ecsv', 'raw.fits',
       'raw.jpg', 'segment-cat.ecsv', 'spt.fits', 'trl.fits', 'trl.txt'],
      dtype='<U16')

In [19]:
pmask = [('drz.fits' in prod) | ('drc.fits' in prod) | ('flc.fits' in prod) for prod in prlst['productFilename']]

In [20]:
prlst[pmask]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str8,str3,str5,str35,str64,str1,str67,str9,str28,str11,str1,str7,str20,str5,str50,int64,str8,str6,int64,str9
24109735,HST,image,jbqz25gqq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gqq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gqq_flc.fits,168327360,24835771,PUBLIC,2,F814W
24109736,HST,image,jbqz25grq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25grq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25grq_flc.fits,168327360,24835771,PUBLIC,2,F814W
24109737,HST,image,jbqz25gtq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gtq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gtq_flc.fits,168327360,24835771,PUBLIC,2,F814W
24109738,HST,image,jbqz25gvq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gvq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gvq_flc.fits,168327360,24835771,PUBLIC,2,F814W
24835771,HST,image,jbqz25010,DADS DRC file - CTE-corrected calibrated combined image ACS/WFC3,D,mast:HST/product/jbqz25010_drc.fits,SCIENCE,Minimum Recommended Products,DRC,--,CALACS,DrizzlePac 3.6.2,12477,jbqz25010_drc.fits,224634240,24835771,PUBLIC,3,F814W
24835771,HST,image,jbqz25010,DADS DRZ file - Calibrated combined image ACS/WFC3/WFPC2/STIS,D,mast:HST/product/jbqz25010_drz.fits,SCIENCE,Minimum Recommended Products,DRZ,--,CALACS,DrizzlePac 3.6.2,12477,jbqz25010_drz.fits,224631360,24835771,PUBLIC,3,F814W
26445460,HST,image,hst_12477_25_acs_wfc_f814w_jbqz25gv,HAP fits science image,D,mast:HST/product/hst_12477_25_acs_wfc_f814w_jbqz25gv_drc.fits,SCIENCE,--,DRC,--,HAP-SVM,DrizzlePac 3.6.2,12477,hst_12477_25_acs_wfc_f814w_jbqz25gv_drc.fits,317488320,24835771,PUBLIC,2,F814W
26445460,HST,image,hst_12477_25_acs_wfc_f814w_jbqz25gv,HAP flat-field product,D,mast:HST/product/hst_12477_25_acs_wfc_f814w_jbqz25gv_flc.fits,SCIENCE,--,FLC,--,HAP-SVM,DrizzlePac 3.6.2,12477,hst_12477_25_acs_wfc_f814w_jbqz25gv_flc.fits,168448320,24835771,PUBLIC,2,F814W
26445490,HST,image,hst_12477_25_acs_wfc_f814w_jbqz25gq,HAP fits science image,D,mast:HST/product/hst_12477_25_acs_wfc_f814w_jbqz25gq_drc.fits,SCIENCE,--,DRC,--,HAP-SVM,DrizzlePac 3.6.2,12477,hst_12477_25_acs_wfc_f814w_jbqz25gq_drc.fits,317488320,24835771,PUBLIC,2,F814W


In [21]:
productlist = prlst[pmask]

In [22]:
downloadFilenames = []
for prod in productlist:
    filename = prod['productFilename']

    # Cut down new HST filenames that start with hst_PROGID
    filename = '_'.join(filename.split('_')[-2:])
    downloadFilenames.append(filename)

productlist.add_column(Column(downloadFilenames, name='downloadFilename'))

# Check that all files to download are unique
if productlist and len(productlist)>1:
    productlist = unique(productlist, keys='downloadFilename')

# Sort by obsID in case we need to reference
productlist.sort('obsID')

In [23]:
productlist

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters,downloadFilename
str8,str3,str5,str35,str64,str1,str67,str9,str28,str11,str1,str7,str20,str5,str50,int64,str8,str6,int64,str9,str18
24109735,HST,image,jbqz25gqq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gqq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gqq_flc.fits,168327360,24835771,PUBLIC,2,F814W,jbqz25gqq_flc.fits
24109736,HST,image,jbqz25grq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25grq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25grq_flc.fits,168327360,24835771,PUBLIC,2,F814W,jbqz25grq_flc.fits
24109737,HST,image,jbqz25gtq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gtq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gtq_flc.fits,168327360,24835771,PUBLIC,2,F814W,jbqz25gtq_flc.fits
24109738,HST,image,jbqz25gvq,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/jbqz25gvq_flc.fits,SCIENCE,--,FLC,--,CALACS,10.3.5 (08-Feb-2022),12477,jbqz25gvq_flc.fits,168327360,24835771,PUBLIC,2,F814W,jbqz25gvq_flc.fits
24835771,HST,image,jbqz25010,DADS DRC file - CTE-corrected calibrated combined image ACS/WFC3,D,mast:HST/product/jbqz25010_drc.fits,SCIENCE,Minimum Recommended Products,DRC,--,CALACS,DrizzlePac 3.6.2,12477,jbqz25010_drc.fits,224634240,24835771,PUBLIC,3,F814W,jbqz25010_drc.fits
24835771,HST,image,jbqz25010,DADS DRZ file - Calibrated combined image ACS/WFC3/WFPC2/STIS,D,mast:HST/product/jbqz25010_drz.fits,SCIENCE,Minimum Recommended Products,DRZ,--,CALACS,DrizzlePac 3.6.2,12477,jbqz25010_drz.fits,224631360,24835771,PUBLIC,3,F814W,jbqz25010_drz.fits
26445460,HST,image,hst_12477_25_acs_wfc_f814w_jbqz25gv,HAP fits science image,D,mast:HST/product/hst_12477_25_acs_wfc_f814w_jbqz25gv_drc.fits,SCIENCE,--,DRC,--,HAP-SVM,DrizzlePac 3.6.2,12477,hst_12477_25_acs_wfc_f814w_jbqz25gv_drc.fits,317488320,24835771,PUBLIC,2,F814W,jbqz25gv_drc.fits
26445460,HST,image,hst_12477_25_acs_wfc_f814w_jbqz25gv,HAP flat-field product,D,mast:HST/product/hst_12477_25_acs_wfc_f814w_jbqz25gv_flc.fits,SCIENCE,--,FLC,--,HAP-SVM,DrizzlePac 3.6.2,12477,hst_12477_25_acs_wfc_f814w_jbqz25gv_flc.fits,168448320,24835771,PUBLIC,2,F814W,jbqz25gv_flc.fits
26445490,HST,image,hst_12477_25_acs_wfc_f814w_jbqz25gq,HAP fits science image,D,mast:HST/product/hst_12477_25_acs_wfc_f814w_jbqz25gq_drc.fits,SCIENCE,--,DRC,--,HAP-SVM,DrizzlePac 3.6.2,12477,hst_12477_25_acs_wfc_f814w_jbqz25gq_drc.fits,317488320,24835771,PUBLIC,2,F814W,jbqz25gq_drc.fits


#### general productlist for all observations

In [19]:
obs = obsTable_

In [20]:
# for i, obs in enumerate(obsTable_):
#     print('- '*10 + str(i) + ' -'*10)
try:
    productList = Observations.get_product_list(obs)
    # Ignore the 'C' type products
    mask = productList['type']=='S'
    productList = productList[mask]
except:
    error = 'ERROR: MAST is not working currently working\n'
    error += 'Try again later...'
    print(error)
#         return(productlist)

instrument = obs['instrument_name']
s_ra = obs['s_ra']
s_dec = obs['s_dec']

instcol = Column([instrument]*len(productList), name='instrument_name')
racol = Column([s_ra]*len(productList), name='ra')
deccol = Column([s_dec]*len(productList), name='dec')

productList.add_column(instcol)
productList.add_column(racol)
productList.add_column(deccol)

for prod in productList:
    filename = prod['productFilename']

    if (('c0m.fits' in filename and 'WFPC2' in instrument) or
        ('c1m.fits' in filename and 'WFPC2' in instrument) or
        ('c0m.fits' in filename and 'PC/WFC' in instrument) or
        ('c1m.fits' in filename and 'PC/WFC' in instrument) or
        ('flc.fits' in filename and 'ACS/WFC' in instrument) or
        ('flt.fits' in filename and 'ACS/HRC' in instrument) or
        ('flc.fits' in filename and 'WFC3/UVIS' in instrument) or
        ('flt.fits' in filename and 'WFC3/IR' in instrument)):

        if not productlist:
            productlist = Table(prod)
        else:
            productlist.add_row(prod)

In [21]:
productlist

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters,instrument_name,ra,dec
str8,str3,str5,str43,str64,str1,str69,str9,str28,str11,str1,str7,str19,str5,str52,int64,str8,str6,int64,str5,str9,float64,float64
23956109,HST,image,icn105e5q,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/icn105e5q_flc.fits,SCIENCE,--,FLC,--,CALWF3,3.7.1 (Oct-18-2023),13677,icn105e5q_flc.fits,169148160,26617203,PUBLIC,2,F814W,WFC3/UVIS,31.4428,-58.4852


In [22]:
downloadFilenames = []
for prod in productlist:
    filename = prod['productFilename']

    # Cut down new HST filenames that start with hst_PROGID
    filename = '_'.join(filename.split('_')[-2:])
    downloadFilenames.append(filename)

productlist.add_column(Column(downloadFilenames, name='downloadFilename'))

# Check that all files to download are unique
if productlist and len(productlist)>1:
    productlist = unique(productlist, keys='downloadFilename')

# Sort by obsID in case we need to reference
productlist.sort('obsID')

In [23]:
productlist

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters,instrument_name,ra,dec,downloadFilename
str8,str3,str5,str43,str64,str1,str69,str9,str28,str11,str1,str7,str19,str5,str52,int64,str8,str6,int64,str5,str9,float64,float64,str18
23956109,HST,image,icn105e5q,DADS FLC file - CTE-corrected calibrated exposure ACS/WFC3,S,mast:HST/product/icn105e5q_flc.fits,SCIENCE,--,FLC,--,CALWF3,3.7.1 (Oct-18-2023),13677,icn105e5q_flc.fits,169148160,26617203,PUBLIC,2,F814W,WFC3/UVIS,31.4428,-58.4852,icn105e5q_flc.fits


In [24]:
Observations.download_products(productlist, extension='fits')

Local Path,Status,Message,URL
str47,str8,object,object
./mastDownload/HST/icn105e5q/icn105e5q_flc.fits,COMPLETE,None,None


In [4]:
workdir = '.'
input_images = [s for p in ['*flc.fits', '*flt.fits'] for s in glob.glob(os.path.join(workdir, p))]

In [5]:
input_images

['./jbqz24tyq_flc.fits',
 './jbqz25grq_flc.fits',
 './icn105e5q_flc.fits',
 './jbqz23ttq_flc.fits',
 './jbqz23trq_flc.fits',
 './jbqz25gtq_flc.fits',
 './jbqz24u1q_flc.fits',
 './jbqz24tzq_flc.fits',
 './jbqz22skq_flc.fits',
 './jbqz25gqq_flc.fits',
 './jbqz22soq_flc.fits',
 './jbqz23tvq_flc.fits',
 './jbqz25gvq_flc.fits',
 './jbqz22siq_flc.fits',
 './jbqz23tqq_flc.fits',
 './jbqz24u3q_flc.fits',
 './jbqz22smq_flc.fits',
 './icn105dvq_flt.fits',
 './icn105dwq_flt.fits']

In [13]:
img = input_images
zptype = 'abmag'
good = []
image_number = []
#check if image exists (check in archive otherwise)
for image in img:
    success = True
    if not os.path.exists(image):
        success = False
    if success:
        good.append(image)
img = copy.copy(good)

hdu = fits.open(img[0])
h = hdu[0].header

exp = [fits.getval(image,'EXPTIME') for image in img] #exposure time
if 'DATE-OBS' in h.keys() and 'TIME-OBS' in h.keys(): 
    dat = [fits.getval(image,'DATE-OBS') + 'T' +
           fits.getval(image,'TIME-OBS') for image in img] #datetime
elif 'EXPSTART' in h.keys():
    dat = [Time(fits.getval(image, 'EXPSTART'),
        format='mjd').datetime.strftime('%Y-%m-%dT%H:%M:%S') #datetime if DATE-OBS is missing
        for image in img]
    
fil = [get_filter(image) for image in img]
ins = [get_instrument(image) for image in img]
det = ['_'.join(get_instrument(image).split('_')[:2]) for image in img]
chip= [get_chip(image) for image in img]
zpt = [get_zpt(i, ccdchip=c, zptype=zptype) for i,c in zip(img,chip)]

if not image_number:
    image_number = [0 for image in img]
    
obstable = Table([img,exp,dat,fil,ins,det,zpt,chip,image_number],
    names=['image','exptime','datetime','filter','instrument',
     'detector','zeropoint','chip','imagenumber'])
obstable.sort('datetime')
obstable = add_visit_info(obstable)

In [14]:
obstable.add_column(Column([' '*99]*len(obstable), name='drizname'))
for i,row in enumerate(obstable):
    visit = row['visit']
    n = str(visit).zfill(4)
    inst = row['instrument']
    filt = row['filter']

    # Visit should correspond to first image so they're all the same
    visittable = obstable[obstable['visit']==visit]
    refimage = visittable['image'][0]
    if 'DATE-OBS' in h.keys():
        date_obj = Time(fits.getval(refimage, 'DATE-OBS'))
    else:
        date_obj = Time(fits.getval(refimage, 'EXPSTART'), format='mjd')
    date_str = date_obj.datetime.strftime('%y%m%d')

    # Make a photpipe-like image name
    drizname = ''
    objname = None
    if objname:
        drizname = '{obj}.{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str, obj=objname)
    else:
        drizname = '{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str)

    if '.':
        drizname = os.path.join('.', drizname)

    obstable[i]['drizname'] = drizname

In [15]:
obstable

image,exptime,datetime,filter,instrument,detector,zeropoint,chip,imagenumber,visit,drizname
str20,float64,str19,str5,str14,str9,float64,str21,int64,int64,str99
./jbqz22siq_flc.fits,480.0,2012-01-04T08:08:04,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22skq_flc.fits,480.0,2012-01-04T08:18:42,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22smq_flc.fits,480.0,2012-01-04T08:29:20,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22soq_flc.fits,480.0,2012-01-04T08:39:58,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tqq_flc.fits,480.0,2012-01-04T16:06:48,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23trq_flc.fits,480.0,2012-01-04T16:17:26,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23ttq_flc.fits,480.0,2012-01-04T16:28:04,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tvq_flc.fits,480.0,2012-01-04T16:38:42,f606w,acs_wfc_full,acs_wfc,26.491377054545822,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz24tyq_flc.fits,480.0,2012-01-04T18:08:11,f606w,acs_wfc_full,acs_wfc,26.491376957830788,1,0,1,./acs.f606w.ut120104_0001.drz.fits


In [16]:
table = organize_reduction_tables(obstable, byvisit = True)

In [17]:
table

[<Table length=12>
        image         exptime ... visit              drizname             
        str20         float64 ... int64               str99               
 -------------------- ------- ... ----- ----------------------------------
 ./jbqz22siq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz22skq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz22smq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz22soq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz23tqq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz23trq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz23ttq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz23tvq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz24tyq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbqz24tzq_flc.fits   480.0 ...     1 ./acs.f606w.ut120104_0001.drz.fits
 ./jbq

### JWST

In [2]:
ra, dec = 288.26458333, 19.77305556
coord = SkyCoord(ra, dec, frame='icrs', unit='deg')
radius = 5*u.arcmin

In [3]:
obsTable = Observations.query_region(coord, radius=radius)

In [4]:
obsTable = obsTable.filled()

In [5]:
#obsTable masks
masks = []
masks.append([t.upper()=='JWST' for t in obsTable['obs_collection']]) #JWST images
masks.append([any(l) for l in list(map(list,zip(*[[det in inst.upper() #NIRCAM images
            for inst in obsTable['instrument_name']]
            for det in ['NIRCAM']])))])

#add mask to remove entries with 1. null jpegURL/dataURL 2. private data rights(?)
# Added mask to remove calibration data from search
masks.append([f.upper()!='DETECTION' for f in obsTable['filters']])
masks.append([i.upper()!='CALIBRATION' for i in obsTable['intentType']])

mask = [all(l) for l in list(map(list, zip(*masks)))]
obsTable_webb = obsTable[mask]

In [6]:
obsTable_webb

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,distance
str11,str5,str7,str13,str4,str12,str16,str13,str45,str65,float64,float64,str10,str19,int64,float64,float64,float64,float64,float64,str100,float64,str23,str8,int64,str8379,str120,str125,str6,bool,float64,str9,float64
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F115W,Infrared,GRB221009A-OT,Star; Gamma Ray transients; Supernovae,jw02784-o002_t001_nircam_clear-f115w,288.26459166666666,19.773405555555556,image,"Blanchard, Peter",3,60056.29769903032,60056.30726767361,558.312,1013.0000000000001,1282.0,Late Time Observations of GRB 221009A: The First Search for r-Process Nucleosynthesis in a Collapsar,60057.27241895,2784,DD,999999,POLYGON 288.29566713 19.771911515 288.257749326 19.76239347 288.247577599 19.798257155 288.285503334 19.807777343,mast:JWST/product/jw02784-o002_t001_nircam_clear-f115w_i2d.jpg,mast:JWST/product/jw02784-o002_t001_nircam_clear-f115w_i2d.fits,PUBLIC,False,nan,128929603,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,Infrared,GRB221009A-OT,Star; Gamma Ray transients; Supernovae,jw02784-o002_t001_nircam_clear-f200w,288.26459166666666,19.773405555555556,image,"Blanchard, Peter",3,60056.30975309282,60056.31932173611,558.312,1755.0,2226.0,Late Time Observations of GRB 221009A: The First Search for r-Process Nucleosynthesis in a Collapsar,60057.27241895,2784,DD,999999,POLYGON 288.295654827 19.771896715 288.257735538 19.762378385 288.247563506 19.798243477 288.285490727 19.807763951,mast:JWST/product/jw02784-o002_t001_nircam_clear-f200w_i2d.jpg,mast:JWST/product/jw02784-o002_t001_nircam_clear-f200w_i2d.fits,PUBLIC,False,nan,128929606,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F277W,Infrared,GRB221009A-OT,Star; Gamma Ray transients; Supernovae,jw02784-o002_t001_nircam_clear-f277w,288.26459166666666,19.773405555555556,image,"Blanchard, Peter",3,60056.297698289585,60056.30726767361,558.312,2416.0,3127.0,Late Time Observations of GRB 221009A: The First Search for r-Process Nucleosynthesis in a Collapsar,60057.27241895,2784,DD,999999,POLYGON 288.295352431 19.77197896 288.258176336 19.762438004 288.247997687 19.797540334 288.285181367 19.807083392,mast:JWST/product/jw02784-o002_t001_nircam_clear-f277w_i2d.jpg,mast:JWST/product/jw02784-o002_t001_nircam_clear-f277w_i2d.fits,PUBLIC,False,nan,128929605,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F444W,Infrared,GRB221009A-OT,Star; Gamma Ray transients; Supernovae,jw02784-o002_t001_nircam_clear-f444w,288.26459166666666,19.773405555555556,image,"Blanchard, Peter",3,60056.30975309282,60056.31932173611,558.312,3880.0,4986.0,Late Time Observations of GRB 221009A: The First Search for r-Process Nucleosynthesis in a Collapsar,60057.27241895,2784,DD,999999,POLYGON 288.295360397 19.771979169 288.258199237 19.762442135 288.248019861 19.797547304 288.285188603 19.807086441,mast:JWST/product/jw02784-o002_t001_nircam_clear-f444w_i2d.jpg,mast:JWST/product/jw02784-o002_t001_nircam_clear-f444w_i2d.fits,PUBLIC,False,nan,128929604,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F115W,Infrared,GRB221009A-OT,Star; Gamma Ray transients; Supernovae,jw02784-o003_t001_nircam_clear-f115w,288.26459166666666,19.773405555555556,image,"Blanchard, Peter",3,60191.81065919039,60191.83066708333,1460.2,1013.0000000000001,1282.0,Late Time Observations of GRB 221009A: The First Search for r-Process Nucleosynthesis in a Collapsar,60192.00099532,2784,DD,999999,POLYGON 288.24046956 19.791960285 288.277007979 19.778466063 288.262597337 19.743918521 288.226065616 19.757409819,mast:JWST/product/jw02784-o003_t001_nircam_clear-f115w_i2d.jpg,mast:JWST/product/jw02784-o003_t001_nircam_clear-f115w_i2d.fits,PUBLIC,False,nan,173643342,0.0
science,JWST,CALJWST,NIRCAM/IMAGE,JWST,F200W,Infrare

In [7]:
productList_0 = Observations.get_product_list(obsTable_webb[0])

In [16]:
productList = productList_0
stage = 2

In [17]:
#product list masks
productmasks = []
productmasks.append([p.upper() == 'SCIENCE' for p in productList['productType']])
if stage == 2:
    productmasks.append([t == 'CAL' for t in productList['productSubGroupDescription']])
    productmasks.append([c == 2 for c in productList['calib_level']])
if stage == 3:
    productmasks.append([t == 'I2D' for t in productList['productSubGroupDescription']])
    productmasks.append([c == 3 for c in productList['calib_level']])


productmask = [all(l) for l in list(map(list, zip(*productmasks)))]
productList = productList[productmask]

In [18]:
productList

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str68,str9,str28,str11,str1,str7,str6,str4,str50,int64,str9,str6,int64,str5
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00001_nrcb1_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00001_nrcb1_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908654,JWST,image,jw02784002001_02101_00003_nrcb3,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00003_nrcb3_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00003_nrcb3_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908665,JWST,image,jw02784002001_02101_00001_nrcb3,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00001_nrcb3_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00001_nrcb3_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908670,JWST,image,jw02784002001_02101_00002_nrcb2,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00002_nrcb2_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00002_nrcb2_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908703,JWST,image,jw02784002001_02101_00001_nrcb4,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00001_nrcb4_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00001_nrcb4_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908708,JWST,image,jw02784002001_02101_00004_nrcb4,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00004_nrcb4_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00004_nrcb4_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908714,JWST,image,jw02784002001_02101_00003_nrcb1,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00003_nrcb1_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00003_nrcb1_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908730,JWST,image,jw02784002001_02101_00002_nrcb3,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00002_nrcb3_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00002_nrcb3_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908735,JWST,image,jw02784002001_02101_00004_nrcb2,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00004_nrcb2_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00004_nrcb2_cal.fits,117538560,128929603,PUBLIC,2,F115W


In [15]:
Observations.download_products(productList, extension='fits')

Local Path,Status,Message,URL
str102,str8,object,object
./mastDownload/JWST/jw02784-o002_t001_nircam_clear-f115w/jw02784-o002_t001_nircam_clear-f115w_i2d.fits,COMPLETE,None,None


In [31]:
productList_0

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str68,str9,str28,str11,str1,str7,str6,str4,str50,int64,str9,str6,int64,str5
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star fine guide,S,mast:JWST/product/jw02784002001_gs-fg_2023112073621_cal.fits,AUXILIARY,--,GS-FG,--,CALJWST,1.13.3,2784,jw02784002001_gs-fg_2023112073621_cal.fits,3093120,128929603,PUBLIC,2,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star fine guide,S,mast:JWST/product/jw02784002001_gs-fg_2023112072259_cal.fits,AUXILIARY,--,GS-FG,--,CALJWST,1.13.3,2784,jw02784002001_gs-fg_2023112072259_cal.fits,3147840,128929603,PUBLIC,2,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star tracking,S,mast:JWST/product/jw02784002001_gs-track_2023112073321_cal.fits,AUXILIARY,--,GS-TRACK,--,CALJWST,1.13.3,2784,jw02784002001_gs-track_2023112073321_cal.fits,2459520,128929603,PUBLIC,2,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star fine guide,S,mast:JWST/product/jw02784002001_gs-fg_2023112072259_uncal.fits,AUXILIARY,--,GS-FG,--,CALJWST,--,2784,jw02784002001_gs-fg_2023112072259_uncal.fits,4017600,128929603,PUBLIC,1,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star fine guide,S,mast:JWST/product/jw02784002001_gs-fg_2023112071128_stream.fits,AUXILIARY,--,GS-FG,--,CALJWST,--,2784,jw02784002001_gs-fg_2023112071128_stream.fits,5256000,128929603,PUBLIC,1,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star fine guide,S,mast:JWST/product/jw02784002001_gs-fg_2023112071859_stream.fits,AUXILIARY,--,GS-FG,--,CALJWST,--,2784,jw02784002001_gs-fg_2023112071859_stream.fits,3159360,128929603,PUBLIC,1,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS2 guide star acquisition,S,mast:JWST/product/jw02784002001_gs-acq2_2023112070449_cal.fits,AUXILIARY,--,GS-ACQ2,--,CALJWST,1.13.3,2784,jw02784002001_gs-acq2_2023112070449_cal.fits,106560,128929603,PUBLIC,2,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star fine guide,S,mast:JWST/product/jw02784002001_gs-fg_2023112071515_stream.fits,AUXILIARY,--,GS-FG,--,CALJWST,--,2784,jw02784002001_gs-fg_2023112071515_stream.fits,3159360,128929603,PUBLIC,1,F115W
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,FGS guide star tracking,S,mast:JWST/product/jw02784002001_gs-track_2023112071945_cal.fits,AUXILIARY,--,GS-TRACK,--,CALJWST,1.13.3,2784,jw02784002001_gs-track_2023112071945_cal.fits,2465280,128929603,PUBLIC,2,F115W


In [23]:
productList_0[(productList_0['productType'] == 'SCIENCE') & (productList_0['productSubGroupDescription'] == 'CAL')]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level,filters
str9,str4,str5,str36,str64,str1,str68,str9,str28,str11,str1,str7,str6,str4,str50,int64,str9,str6,int64,str5
128908627,JWST,image,jw02784002001_02101_00001_nrcb1,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00001_nrcb1_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00001_nrcb1_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908654,JWST,image,jw02784002001_02101_00003_nrcb3,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00003_nrcb3_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00003_nrcb3_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908665,JWST,image,jw02784002001_02101_00001_nrcb3,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00001_nrcb3_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00001_nrcb3_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908670,JWST,image,jw02784002001_02101_00002_nrcb2,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00002_nrcb2_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00002_nrcb2_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908703,JWST,image,jw02784002001_02101_00001_nrcb4,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00001_nrcb4_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00001_nrcb4_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908708,JWST,image,jw02784002001_02101_00004_nrcb4,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00004_nrcb4_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00004_nrcb4_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908714,JWST,image,jw02784002001_02101_00003_nrcb1,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00003_nrcb1_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00003_nrcb1_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908730,JWST,image,jw02784002001_02101_00002_nrcb3,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00002_nrcb3_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00002_nrcb3_cal.fits,117538560,128929603,PUBLIC,2,F115W
128908735,JWST,image,jw02784002001_02101_00004_nrcb2,exposure (L2b): 2D calibrated exposure average over integrations,S,mast:JWST/product/jw02784002001_02101_00004_nrcb2_cal.fits,SCIENCE,--,CAL,--,CALJWST,1.13.3,2784,jw02784002001_02101_00004_nrcb2_cal.fits,117538560,128929603,PUBLIC,2,F115W


In [22]:
np.unique(productList_0['productSubGroupDescription'])

ANNNN_CRF
ASN
CAL
CAT
GS-ACQ1
GS-ACQ2
GS-FG
GS-ID
GS-TRACK
I2D
--


In [12]:
productList_0.colnames

['obsID',
 'obs_collection',
 'dataproduct_type',
 'obs_id',
 'description',
 'type',
 'dataURI',
 'productType',
 'productGroupDescription',
 'productSubGroupDescription',
 'productDocumentationURL',
 'project',
 'prvversion',
 'proposal_id',
 'productFilename',
 'size',
 'parent_obsid',
 'dataRights',
 'calib_level',
 'filters']

In [65]:
for i, a in enumerate(np.arange(1, 5, 1)):
    print(str(i).zfill(4))

0000
0001
0002
0003


#### Misc

In [ ]:
def needs_to_be_reduced(self, image, save_c1m=False):

    keep_short = self.options['args'].keep_short
    keep_tdf_down = self.options['args'].keep_tdf_down
    keep_indt = self.options['args'].keep_indt

    #same for jwst
    if not os.path.exists(image):
        success = self.try_to_get_image(image)
        if not success:
            warning = 'WARNING: {image} does not exist'
            return(warning.format(img=image), False)

    #same for jwst
    try:
        hdu = fits.open(image, mode='readonly')
        check = False
        for h in hdu:
            if h.data is not None and h.name.upper()=='SCI':
                check = True
    except (OSError, TypeError, AttributeError):
        warning = 'WARNING: {img} is empty or corrupted.  '
        warning += 'Trying to download again...'
        print(warning.format(img=image))

        success = False
        if not self.productlist:
            warning = 'WARNING: could not find or download {img}'
            return(warning.format(img=image), False)

        mask = self.productlist['productFilename']==image
        if self.productlist[mask]==0: #len?
            warning = 'WARNING: could not find or download {img}'
            return(warning.format(img=image), False)

        self.download_files(self.productlist,
            archivedir=self.options['args'].archive, 
            clobber=True)

        for product in self.productlist[mask]:
            self.copy_raw_data_archive(product, 
                archivedir=self.options['args'].archive,
                workdir=self.options['args'].work_dir, 
                check_for_coord=True)

        if os.path.exists(image):
            try:
                hdu = fits.open(image, mode='readonly')
                check = False
                for h in hdu:
                    if h.data is not None and h.name.upper()=='SCI':
                        check = True
            except (OSError, TypeError, AttributeError):
                warning = 'WARNING: could not find or download {img}'
                return(warning.format(img=image), False)

    is_not_hst_image = False
    warning = ''
    detector = ''

    # Check for header keys that we need
    for key in ['INSTRUME','EXPTIME','DATE-OBS','TIME-OBS']:
        if key not in hdu[0].header.keys():
            warning = 'WARNINGS: {key} not in {img} header'
            warning = warning.format(key=key, img=image)
            return(warning, False)

    instrument = hdu[0].header['INSTRUME'].lower()
    if 'c1m.fits' in image and not save_c1m: # not needed for jwst
        # We need the c1m.fits files, but they aren't reduced as science data
        warning = 'WARNING: do not need to reduce c1m.fits files.'
        return(warning, False)

    if ('DETECTOR' in hdu[0].header.keys()):
        detector = hdu[0].header['DETECTOR'].lower()

    #check expflag for jwst
    # Check for EXPFLAG=='INDETERMINATE', usually indicating a bad exposure
    if 'EXPFLAG' in hdu[0].header.keys():
        flag = hdu[0].header['EXPFLAG']
        if flag=='INDETERMINATE':
            if not keep_indt:
                warning = f'WARNING: {image} has EXPFLAG==INDETERMINATE'
                return(warning, False)
        elif 'TDF-DOWN' in flag:
            if not keep_tdf_down:
                warning = f'WARNING: {image} has EXPFLAG==TDF-DOWN AT EXPSTART'
                return(warning, False)
        elif flag!='NORMAL':
            warning = f'WARNING: {image} has EXPFLAG=={flag}.'
            return(warning, False)

    # Get rid of exposures with exptime < 20s
    if not keep_short:
        exptime = hdu[0].header['EXPTIME']
        if (exptime < 15):
            warning = f'WARNING: {image} EXPTIME is {exptime} < 20.'
            return(warning, False)

    # Now check date and compare to self.before
    mjd_obs = Time(hdu[0].header['DATE-OBS']+'T'+hdu[0].header['TIME-OBS']).mjd
    if self.before is not None:
        mjd_before = Time(self.before).mjd
        dbefore = self.before.strftime('%Y-%m-%d')
        if mjd_obs > mjd_before:
            warning = f'WARNING: {image} after the input before date {dbefore}.'
            return(warning, False)

    # Same with self.after
    if self.after is not None:
        mjd_after = Time(self.after).mjd
        dafter = self.after.strftime('%Y-%m-%d')
        if mjd_obs < mjd_after:
            warning = f'WARNING: {image} before the input after date {dafter}.'
            return(warning, False)

    # Get rid of data where input coordinate not in any extension
    if self.coord:
        for h in hdu:
            if h.data is not None and h.name.upper()=='SCI':
                # This method doesn't need to be very precise and fails if
                # certain variables (e.g., distortion terms) are missing, so
                # construct a very basic dummy header with base terms
                dummy_header = {'CTYPE1': 'RA---TAN', 'CTYPE2': 'DEC--TAN',
                        'CRPIX1': h.header['CRPIX1'],
                        'CRPIX2': h.header['CRPIX2'],
                        'CRVAL1': h.header['CRVAL1'],
                        'CRVAL2': h.header['CRVAL2'],
                        'CD1_1': h.header['CD1_1'], 'CD1_2': h.header['CD1_2'],
                        'CD2_1': h.header['CD2_1'], 'CD2_2': h.header['CD2_2']}
                w = wcs.WCS(dummy_header)
                # This can be rough and sometimes WCS will choke on images
                # if mode='all', so using mode='wcs' (only core WCS)
                x,y = wcs.utils.skycoord_to_pixel(self.coord, w,
                                                      origin=1, mode='wcs')
                if (x > 0 and y > 0 and
                    x < h.header['NAXIS1'] and y < h.header['NAXIS2']):
                    is_not_hst_image = True

        if not is_not_hst_image:
            ra = self.coord.ra.degree
            dec = self.coord.dec.degree
            warning = f'WARNING: {image} does not contain: {ra} {dec}'
            return(warning, False)

    filt = self.get_filter(image).upper()
    if not (filt in self.options['acceptable_filters']):
        warning = f'WARNING: {image} with FILTER={filt} '
        warning += 'does not have an acceptable filter.'
        return(warning, False)

    # we only need nircam (cal/i2d) for jwst
    # Get rid of images that don't match one of the allowed instrument/detector
    # types and images whose extensions don't match the allowed type for those
    # instrument/detector types
    is_not_hst_image = False
    nextend = hdu[0].header['NEXTEND']
    warning = f'WARNING: {image} with INSTRUME={instrument}, '
    warning += f'DETECTOR={detector}, NEXTEND={nextend} is bad.'
    if (instrument.upper() == 'WFPC2' and 'c0m.fits' in image and nextend==4):
        is_not_hst_image = True
    if (instrument.upper() == 'ACS' and
        detector.upper() == 'WFC' and 'flc.fits' in image):
        is_not_hst_image = True
    if (instrument.upper() == 'ACS' and
        detector.upper() == 'HRC' and 'flt.fits' in image):
        is_not_hst_image = True
    if (instrument.upper() == 'WFC3' and
        detector.upper() == 'UVIS' and 'flc.fits' in image):
        is_not_hst_image = True
    if (instrument.upper() == 'WFC3' and
        detector.upper() == 'IR' and 'flt.fits' in image):
        is_not_hst_image = True
    if save_c1m:
        if (instrument.upper() == 'WFPC2' and 'c1m.fits' in image):
            is_not_hst_image = True

    return(warning, is_not_hst_image)

In [ ]:
def input_list(img, show=True, save=False, file=None, image_number=[])
    zptype = 'abmag'

    # To prevent FileNotFoundError - make sure all images exist and if not then
    # try to download them
    good = []
    for image in img:
        success = True
        if not os.path.exists(image):
            success = self.try_to_get_image(image)
        if success:
            good.append(image)

    if not good:
        return(None)
    else:
        img = copy.copy(good)

    hdu = fits.open(img[0])
    h = hdu[0].header

    # Make a table with all of the metadata for each image.
    exp = [fits.getval(image,'EXPTIME') for image in img]
    if 'DATE-OBS' in h.keys() and 'TIME-OBS' in h.keys():
        dat = [fits.getval(image,'DATE-OBS') + 'T' +
               fits.getval(image,'TIME-OBS') for image in img]
    # This should work if image is missing DATE-OBS, e.g., for drz images
    elif 'EXPSTART' in h.keys():
        dat = [Time(fits.getval(image, 'EXPSTART'),
            format='mjd').datetime.strftime('%Y-%m-%dT%H:%M:%S')
            for image in img]
    fil = [self.get_filter(image) for image in img]
    ins = [self.get_instrument(image) for image in img]
    det = ['_'.join(self.get_instrument(image).split('_')[:2]) for image in img]
    chip= [self.get_chip(image) for image in img]
    zpt = [self.get_zpt(i, ccdchip=c, zptype=zptype) for i,c in zip(img,chip)]

    if not image_number:
        image_number = [0 for image in img]

    # Save this obstable.  Useful for other methods
    obstable = Table([img,exp,dat,fil,ins,det,zpt,chip,image_number],
        names=self.names)

    # Look at the table in order of date
    obstable.sort('datetime')

    # Automatically add visit info
    obstable = self.add_visit_info(obstable)

    # Show the obstable in a column formatted style
    form = '{file: <36} {inst: <18} {filt: <10} '
    form += '{exp: <12} {date: <10} {time: <10}'
    if show:
        header = form.format(file='FILE',inst='INSTRUMENT',filt='FILTER',
                             exp='EXPTIME',date='DATE-OBS',time='TIME-OBS')
        print('\n\n')
        print(header)

        for row in obstable:
            line = form.format(file=os.path.basename(row['image']),
                    inst=row['instrument'].upper(),
                    filt=row['filter'].upper(),
                    exp='%7.4f' % row['exptime'],
                    date=Time(row['datetime']).datetime.strftime('%Y-%m-%d'),
                    time=Time(row['datetime']).datetime.strftime('%H:%M:%S'))
            print(line)

        print('\n\n')

    # Iterate over visit, instrument, filter to add group-specific info
    obstable.add_column(Column([' '*99]*len(obstable), name='drizname'))
    for i,row in enumerate(obstable):
        visit = row['visit']
        n = str(visit).zfill(4)
        inst = row['instrument']
        filt = row['filter']

        # Visit should correspond to first image so they're all the same
        visittable = obstable[obstable['visit']==visit]
        refimage = visittable['image'][0]
        if 'DATE-OBS' in h.keys():
            date_obj = Time(fits.getval(refimage, 'DATE-OBS'))
        else:
            date_obj = Time(fits.getval(refimage, 'EXPSTART'), format='mjd')
        date_str = date_obj.datetime.strftime('%y%m%d')

        # Make a photpipe-like image name
        drizname = ''
        objname = self.options['args'].object
        if objname:
            drizname = '{obj}.{inst}.{filt}.ut{date}_{n}.drz.fits'
            drizname = drizname.format(inst=inst.split('_')[0],
                filt=filt, n=n, date=date_str, obj=objname)
        else:
            drizname = '{inst}.{filt}.ut{date}_{n}.drz.fits'
            drizname = drizname.format(inst=inst.split('_')[0],
                filt=filt, n=n, date=date_str)

        if self.options['args'].work_dir:
            drizname = os.path.join(self.options['args'].work_dir, drizname)

        obstable[i]['drizname'] = drizname

    if file:

        form = '{inst: <10} {filt: <10} {exp: <12} {date: <16}'
        header = form.format(inst='INSTRUMENT', filt='FILTER', exp='EXPTIME',
            date='DATE')

        if self.options['args'].work_dir:
            file = os.path.join(self.options['args'].work_dir, file)

        outfile = open(file, 'w')
        outfile.write(header+'\n')

        for visit in list(set(obstable['visit'])):
            visittable = obstable[obstable['visit'] == visit]
            for inst in list(set(visittable['instrument'])):
                insttable = visittable[visittable['instrument'] == inst]
                for filt in list(set(insttable['filter'])):
                    ftable = insttable[insttable['filter'] == filt]

                    # Format instrument name
                    if 'wfc3' in inst and 'uvis' in inst: instname='WFC3/UVIS'
                    if 'wfc3' in inst and 'ir' in inst: instname='WFC3/IR'
                    if 'acs' in inst and 'wfc' in inst: instname='ACS/WFC'
                    if 'acs' in inst and 'hrc' in inst: instname='ACS/HRC'
                    if 'acs' in inst and 'sbc' in inst: instname='ACS/SBC'
                    if 'wfpc2' in inst: instname='WFPC2'
                    if '_' in instname:
                        instname=instname.upper()
                        instname=instname.replace('_full','').replace('_','/')

                    mjd = [Time(r['datetime']).mjd for r in ftable]
                    time = Time(np.mean(mjd), format='mjd')

                    exptime = np.sum(ftable['exptime'])

                    date_decimal='%1.5f'% (time.mjd % 1.0)
                    date = time.datetime.strftime('%Y-%m-%d')
                    date += date_decimal[1:]

                    line=form.format(date=date, inst=instname,
                        filt=filt.upper(), exp='%7.4f' % exptime)
                    outfile.write(line+'\n')

        outfile.close()

    # Save as the primary obstable for this reduction?
    if save:
        self.obstable = obstable

    return(obstable)

#### check visits

In [116]:
inst = np.array(obstable['instrument'])
mjd = Time(obstable['datetime']).mjd
filt = np.array(obstable['filter'])
filename = np.array(obstable['image'])
visit = np.zeros(len(inst))
tol = 1

In [117]:
filt[3] = 'f814w'

In [118]:
for i in range(len(visit)):
    if all([visiti == 0 for visiti in visit]):
        visit[i] = int(1)
    else:
        instmask = list(inst == inst[i])
        timemask = list(abs(mjd - mjd[i]) < tol)
        filtmask = list(filt[i] == filt)
        nzero = list(visit != 0) # Ignore unassigned visits
        mask = [all(l) for l in zip(instmask, timemask, filtmask, nzero)]
#         print(mask)

        # If no matches, then we need to define a new visit
        if not any(mask):
            # no matches. create new visit number = max(visit)+1
            visit[i] = int(np.max(visit) + 1)
        else:
            # otherwise, confirm that all visits of obstable[mask] are same
            if (len(list(set(visit[mask]))) != 1):
                error = 'ERROR: visit numbers are incorrectly assigned.'
                print(error)
#                 return(None)
            else:
                # visit number is equal to other values in set
                visit[i] = list(set(visit[mask]))[0]

In [119]:
visit

array([1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 3., 3., 3., 3., 4.,
       4., 5.])

#### end visits

In [ ]:
def pick_reference(self, obstable):
    # If we haven't defined input images, catch error

    reference_images = pick_deepest_images(list(obstable['image']),
        reffilter=None,
        avoid_wfpc2=False,
        refinst=None)

    if len(reference_images)==0:
        error = 'ERROR: could not pick a reference image'
        print(error)
        return(None)

    best_filt = get_filter(reference_images[0])
    best_inst = get_instrument(reference_images[0]).split('_')[0]

    vnum = np.min(obstable['visit'].data)
    vnum = str(vnum).zfill(4)

    # Generate photpipe-like output name for the drizzled image
    if self.options['args'].object:
        drizname = '{obj}.{inst}.{filt}.ref_{num}.drz.fits'
        drizname = drizname.format(inst=best_inst, filt=best_filt,
            obj=self.options['args'].object, num=vnum)
    else:
        drizname = '{inst}.{filt}.ref_{num}.drz.fits'
        drizname = drizname.format(inst=best_inst, filt=best_filt, num=vnum)

    reference_images = sorted(reference_images)

    if os.path.exists(drizname):
        hdu = fits.open(drizname)

        # Check for NINPUT and INPUT names
        if 'NINPUT' in hdu[0].header.keys() and 'INPUT' in hdu[0].header.keys():
            # Check that NINPUT and INPUT match what we expect
            ninput = len(reference_images)
            str_input = ','.join([s.split('.')[0] for s in reference_images])

            if (hdu[0].header['INPUT'].startswith(str_input) and
                hdu[0].header['NINPUT']==ninput):
                warning='WARNING: drizzled image {drz} exists.\n'
                warning+='Skipping astrodrizzle...'
                print(warning.format(drz=drizname))
                return(drizname)

    message = 'Reference image name will be: {reference}.\n'
    message += 'Generating from input files: {img}\n\n'
    print(message.format(reference=drizname, img=reference_images))

    if self.options['args'].drizzle_add:
        add_images = list(str(self.options['args'].drizzle_add).split(','))
        for image in add_images:
            if os.path.exists(image) and image not in reference_images:
                reference_images.append(image)

    if 'wfpc2' in best_inst and len(obstable)<3:
        reference_images = glob.glob('u*c0m.fits')

    obstable = self.input_list(reference_images, show=True, save=False)
    if not obstable or len(obstable)==0:
        return(None)

    # If number of images is small, try to use imaging from the same instrument
    # and detector for masking
    if len(obstable)<3 and not self.options['args'].no_mask:
        inst = obstable['instrument'][0]
        det = obstable['detector'][0]
        mask = (obstable['instrument']==inst) & (obstable['detector']==det)

        outimage = '{inst}.ref.drz.fits'.format(inst=inst)

        if not opt.skip_tweakreg:
            error, shift_table = self.run_tweakreg(obstable[mask], '')
        self.run_astrodrizzle(obstable[mask], output_name=outimage,
            clean=False, save_fullfile=True)

        # Add cosmic ray mask to static image mask
        if self.options['args'].add_crmask:
            for row in obstable[mask]:
                file = row['image']
                crmasks = glob.glob(file.replace('.fits','*crmask.fits'))

                for i,crmaskfile in enumerate(sorted(crmasks)):
                    crmaskhdu = fits.open(crmaskfile)
                    crmask = crmaskhdu[0].data==0
                    if 'c0m' in file:
                        maskfile = file.split('_')[0]+'_c1m.fits'
                        if os.path.exists(maskfile):
                            maskhdu = fits.open(maskfile)
                            maskhdu[i+1].data[crmask]=4096
                            maskhdu.writeto(maskfile, overwrite=True)
                    else:
                        maskhdu = fits.open(file)
                        if maskhdu[3*i+1].name=='DQ':
                            maskhdu[3*i+1].data[crmask]=4096
                        maskhdu.writeto(file, overwrite=True)

    if not opt.skip_tweakreg:
        error, shift_table = self.run_tweakreg(obstable, '')
    self.run_astrodrizzle(obstable, output_name=drizname, save_fullfile=True)

    return(drizname)

In [50]:
ref_imgs = []
for i, Obstable in enumerate(table):
    ref_imgs.append(pick_deepest_images(list(Obstable['image'])))

In [52]:
ref_imgs = ref_imgs[0]

In [138]:
obstable

image,exptime,datetime,filter,instrument,detector,zeropoint,chip,imagenumber,visit,drizname
str20,float64,str19,str5,str14,str9,float64,str21,int64,int64,str99
./jbqz22siq_flc.fits,480.0,2012-01-04T08:08:04,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22skq_flc.fits,480.0,2012-01-04T08:18:42,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22smq_flc.fits,480.0,2012-01-04T08:29:20,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz22soq_flc.fits,480.0,2012-01-04T08:39:58,f606w,acs_wfc_full,acs_wfc,26.491377676285538,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tqq_flc.fits,480.0,2012-01-04T16:06:48,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23trq_flc.fits,480.0,2012-01-04T16:17:26,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23ttq_flc.fits,480.0,2012-01-04T16:28:04,f606w,acs_wfc_full,acs_wfc,26.491377137444424,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz23tvq_flc.fits,480.0,2012-01-04T16:38:42,f606w,acs_wfc_full,acs_wfc,26.491377054545822,1,0,1,./acs.f606w.ut120104_0001.drz.fits
./jbqz24tyq_flc.fits,480.0,2012-01-04T18:08:11,f606w,acs_wfc_full,acs_wfc,26.491376957830788,1,0,1,./acs.f606w.ut120104_0001.drz.fits


### tweakreg

In [45]:
def tweakreg_error(exception):
    message = '\n\n' + '#'*80 + '\n'
    message += 'WARNING: tweakreg failed: {e}\n'
    message += '#'*80 + '\n'
    print(message.format(e=exception.__class__.__name__))
    print('Error:', exception)
    print('Adjusting thresholds and images...')

In [46]:
# Apply TWEAKSUC header variable if tweakreg was successful
def apply_tweakreg_success(shifts):

    for row in shifts:
        if ~np.isnan(row['xoffset']) and ~np.isnan(row['yoffset']):
            file=row['file']
            if not os.path.exists(file):
                file=row['file']
                print(f'WARNING: {file} does not exist!')
                continue
            hdu = fits.open(file, mode='update')
            hdu[0].header['TWEAKSUC']=1
            hdu.close()

In [ ]:
# Update image wcs using updatewcs routine
def update_image_wcs(self, image, options, use_db=True):

    hdu = fits.open(image, mode='readonly')
    # Check if tweakreg was successfully run.  If so, then skip
    if 'TWEAKSUC' in hdu[0].header.keys() and hdu[0].header['TWEAKSUC']==1:
        return(True)

    # Check for hierarchical alignment.  If image has been shifted with
    # hierarchical alignment, we don't want to shift it again
    if 'HIERARCH' in hdu[0].header.keys() and hdu[0].header['HIERARCH']==1:
        return(True)

    hdu.close()

    message = 'Updating WCS for {file}'
    print(message.format(file=image))

#     self.clear_downloads(self.options['global_defaults'])

    change_keys = ['IDCTAB','DGEOFILE','NPOLEXT','NPOLFILE','D2IMFILE', 'D2IMEXT','OFFTAB']
    inst = get_instrument(image).split('_')[0]
    ref_url = 'jref.old'

    self.fix_hdu_wcs_keys(image, change_keys, ref_url)

    # Usually if updatewcs fails, that means it's already been done
    try:
        updatewcs.updatewcs(image, use_db=use_db)
        hdu = fits.open(image, mode='update')
        message = '\n\nupdatewcs success.  File info:'
        print(message)
        hdu.info()
        hdu.close()
        self.fix_hdu_wcs_keys(image, change_keys, ref_url)
        self.fix_idcscale(image)
        return(True)
    except:
        error = 'ERROR: failed to update WCS for image {file}'
        print(error.format(file=image))
        return(None)

In [58]:
img = sorted(ref_imgs)
zptype = 'abmag'
good = []
image_number = []
#check if image exists (check in archive otherwise)
for image in img:
    success = True
    if not os.path.exists(image):
        success = False
    if success:
        good.append(image)
img = copy.copy(good)

hdu = fits.open(img[0])
h = hdu[0].header

exp = [fits.getval(image,'EXPTIME') for image in img] #exposure time
if 'DATE-OBS' in h.keys() and 'TIME-OBS' in h.keys(): 
    dat = [fits.getval(image,'DATE-OBS') + 'T' +
           fits.getval(image,'TIME-OBS') for image in img] #datetime
elif 'EXPSTART' in h.keys():
    dat = [Time(fits.getval(image, 'EXPSTART'),
        format='mjd').datetime.strftime('%Y-%m-%dT%H:%M:%S') #datetime if DATE-OBS is missing
        for image in img]
    
fil = [get_filter(image) for image in img]
ins = [get_instrument(image) for image in img]
det = ['_'.join(get_instrument(image).split('_')[:2]) for image in img]
chip= [get_chip(image) for image in img]
zpt = [get_zpt(i, ccdchip=c, zptype=zptype) for i,c in zip(img,chip)]

if not image_number:
    image_number = [0 for image in img]
    
obstable = Table([img,exp,dat,fil,ins,det,zpt,chip,image_number],
    names=['image','exptime','datetime','filter','instrument',
     'detector','zeropoint','chip','imagenumber'])
obstable.sort('datetime')
obstable = add_visit_info(obstable)

In [59]:
obstable.add_column(Column([' '*99]*len(obstable), name='drizname'))
for i,row in enumerate(obstable):
    visit = row['visit']
    n = str(visit).zfill(4)
    inst = row['instrument']
    filt = row['filter']

    # Visit should correspond to first image so they're all the same
    visittable = obstable[obstable['visit']==visit]
    refimage = visittable['image'][0]
    if 'DATE-OBS' in h.keys():
        date_obj = Time(fits.getval(refimage, 'DATE-OBS'))
    else:
        date_obj = Time(fits.getval(refimage, 'EXPSTART'), format='mjd')
    date_str = date_obj.datetime.strftime('%y%m%d')

    # Make a photpipe-like image name
    drizname = ''
    objname = None
    if objname:
        drizname = '{obj}.{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str, obj=objname)
    else:
        drizname = '{inst}.{filt}.ut{date}_{n}.drz.fits'
        drizname = drizname.format(inst=inst.split('_')[0],
            filt=filt, n=n, date=date_str)

    if '.':
        drizname = os.path.join('.', drizname)

    obstable[i]['drizname'] = drizname

In [66]:
outdir = '.'
reference = ''
run_images = list(obstable['image'])
shift_table = Table([run_images, [np.nan]*len(run_images), 
                     [np.nan]*len(run_images)], names = ('file', 'xoffset', 'yoffset'))
tmp_images = []

In [ ]:
  # Run tweakreg on all input images
def run_tweakreg(self, obstable, reference, do_cosmic=True, skip_wcs=False,
                 search_radius=None, update_hdr=True):

#     outdir = '.'

#     os.chdir(outdir)

#     # Get options from object
#     options = self.options['global_defaults']
#     # Check if tweakreg has already been run on each image
#     run_images = self.check_images_for_tweakreg(list(obstable['image']))
#     if not run_images: return('tweakreg success', None)
#     if reference in run_images: run_images.remove(reference)

#     # Records what the offsets are for the files run through tweakreg
#     shift_table = Table([run_images,[np.nan]*len(run_images),
#         [np.nan]*len(run_images)], names=('file','xoffset','yoffset'))

#     # Check if we just removed all of the images
#     if not run_images:
#         warning = 'WARNING: All images have been run through tweakreg.'
#         print(warning)
#         return(True)

#     print('Need to run tweakreg for images:')
#     self.input_list(obstable['image'], show=True, save=False)

    tmp_images = []
    for image in run_images:
        if self.updatewcs and not skip_wcs:
            det = '_'.join(self.get_instrument(image).split('_')[:2])
            wcsoptions = self.options['detector_defaults'][det]
            self.update_image_wcs(image, wcsoptions)

        if not do_cosmic:
            tmp_images.append(image)
            continue

        # wfc3_ir doesn't need cosmic clean and assume reference is cleaned
        if (image == reference or 'wfc3_ir' in self.get_instrument(image)):
            message = 'Skipping adjustments for {file} as WFC3/IR or reference'
            print(message.format(file=image))
            tmp_images.append(image)
            continue

        rawtmp = image.replace('.fits','.rawtmp.fits')
        tmp_images.append(rawtmp)

        # Check if rawtmp already exists
        if os.path.exists(rawtmp):
            message = '{file} exists. Skipping...'
            print(message.format(file=rawtmp))
            continue

        # Copy the raw data into a temporary file
        shutil.copyfile(image, rawtmp)

        # Clean cosmic rays so they aren't used for alignment
        inst = self.get_instrument(image).split('_')[0]
        crpars = self.options['instrument_defaults'][inst]['crpars']
        self.run_cosmic(rawtmp, crpars)

    modified = False
    ref_images = self.pick_deepest_images(tmp_images)
    deepest = sorted(ref_images, key=lambda im: fits.getval(im, 'EXPTIME'))[-1]
    if (not reference or reference=='dummy.fits'):
        reference = 'dummy.fits'
        message = 'Copying {deep} to reference dummy.fits'
        print(message.format(deep=deepest))
        shutil.copyfile(deepest, reference)
    elif not self.prepare_reference_tweakreg(reference):
        # Can't use this reference image, just use one of the input
        reference = 'dummy.fits'
        message = 'Copying {deep} to reference dummy.fits'
        print(message.format(deep=deepest))
        shutil.copyfile(deepest, reference)
    else:
        modified = True

    message = 'Tweakreg is executing...'
    print(message)

    start_tweak = time.time()

    tweakreg_success = False
    tweak_img = copy.copy(tmp_images)
    ithresh = self.threshold ; rthresh = self.threshold
    shallow_img = []
    thresh_data = None
    tries = 0

    while (not tweakreg_success and tries < 10):
        tweak_img = self.check_images_for_tweakreg(tweak_img)
        if not tweak_img: break
        if tweak_img:
            # Remove images from tweak_img if they are too shallow
            if shallow_img:
                for img in shallow_img:
                    if img in tweak_img:
                        tweak_img.remove(img)

            if len(tweak_img)==0:
                error = 'ERROR: removed all images as shallow'
                print(error)
                tweak_img = copy.copy(tmp_images)
                tweak_img = self.check_images_for_tweakreg(tweak_img)

            # If we've tried multiple runs and there are images in input
            # list with TWEAKSUC and reference image=dummy.fits, we might need
            # to try a different reference image
            success = list(set(tmp_images) ^ set(tweak_img))
            if tries > 1 and reference=='dummy.fits' and len(success)>0:
                # Make random success image new dummy image
                n = len(success)-1
                shutil.copyfile(success[random.randint(0,n)],'dummy.fits')

            # This estimates what the input threshold should be and cuts
            # out images based on number of detected sources from previous
            # rounds of tweakreg
            message = '\n\nReference image: {ref} \n'
            message += 'Images: {im}'
            print(message.format(ref=reference, im=','.join(tweak_img)))

            # Get deepest image and use threshold from that
            deepest = sorted(tweak_img,
                key=lambda im: fits.getval(im, 'EXPTIME'))[-1]

            if not thresh_data or deepest not in thresh_data['file']:
                inp_data = self.get_tweakreg_thresholds(deepest,
                    options['nbright']*4)
                thresh_data = self.add_thresh_data(thresh_data, deepest,
                    inp_data)
            mask = thresh_data['file']==deepest
            inp_thresh = thresh_data[mask][0]
            print('Getting image threshold...')
            new_ithresh = self.get_best_tweakreg_threshold(inp_thresh,
                options['nbright']*4)

            if not thresh_data or reference not in thresh_data['file']:
                inp_data = self.get_tweakreg_thresholds(reference,
                    options['nbright']*4)
                thresh_data = self.add_thresh_data(thresh_data, reference,
                    inp_data)
            mask = thresh_data['file']==reference
            inp_thresh = thresh_data[mask][0]
            print('Getting reference threshold...')
            new_rthresh = self.get_best_tweakreg_threshold(inp_thresh,
                options['nbright']*4)

            if not rthresh: rthresh = self.threshold
            if not ithresh: ithresh = self.threshold

            # Other input options
            nbright = options['nbright']
            minobj = options['minobj']
            search_rad = int(np.round(options['search_rad']))
            if search_radius: search_rad = search_radius

            rconv = 3.5 ; iconv = 3.5 ; tol = 0.25
            if 'wfc3_ir' in self.get_instrument(reference):
                rconv = 2.5
            if all(['wfc3_ir' in self.get_instrument(i)
                for i in tweak_img]):
                iconv = 2.5 ; tol = 0.6
            if 'wfpc2' in self.get_instrument(reference):
                rconv = 2.5
            if all(['wfpc2' in self.get_instrument(i)
                for i in tweak_img]):
                iconv = 2.5 ; tol = 0.5


            # Don't want to keep trying same thing over and over
            if (new_ithresh>=ithresh or new_rthresh>=rthresh) and tries>1:
                # Decrease the threshold and increase tolerance
                message = 'Decreasing threshold and increasing tolerance...'
                print(message)
                ithresh = np.max([new_ithresh*(0.95**tries), 3.0])
                rthresh = np.max([new_rthresh*(0.95**tries), 3.0])
                tol = tol * 1.3**tries
                search_rad = search_rad * 1.2**tries
            else:
                ithresh = new_ithresh
                rthresh = new_rthresh

            if tries > 7:
                minobj = 7

            message = '\nAdjusting thresholds:\n'
            message += 'Reference threshold={rthresh}\n'
            message += 'Image threshold={ithresh}\n'
            message += 'Tolerance={tol}\n'
            message += 'Search radius={rad}\n'
            print(message.format(ithresh='%2.4f'%ithresh,
                rthresh='%2.4f'%rthresh, tol='%2.4f'%tol,
                rad='%2.4f'%search_rad))

            outshifts = os.path.join(outdir, 'drizzle_shifts.txt')

            try:
                tweakreg.TweakReg(files=tweak_img, refimage=reference,
                    verbose=False, interactive=False, clean=True,
                    writecat=True, updatehdr=update_hdr, reusename=True,
                    rfluxunits='counts', minobj=minobj, wcsname='TWEAK',
                    searchrad=search_rad, searchunits='arcseconds', runfile='',
                    tolerance=tol, refnbright=nbright, nbright=nbright,
                    separation=0.5, residplot='No plot', see2dplot=False,
                    fitgeometry='shift',
                    imagefindcfg = {'threshold': ithresh,
                        'conv_width': iconv, 'use_sharp_round': True},
                    refimagefindcfg = {'threshold': rthresh,
                        'conv_width': rconv, 'use_sharp_round': True},
                    shiftfile=True, outshifts=outshifts)

                # Reset shallow_img list
                shallow_img = []

            except AssertionError as e:
                self.tweakreg_error(e)

                message = 'Re-running tweakreg with shallow images removed:'
                print(message)
                for img in tweak_img:
                    nsources = self.get_nsources(img, ithresh)
                    if nsources < 1000:
                        shallow_img.append(img)

            # Occurs when all images fail alignment
            except TypeError as e:
                self.tweakreg_error(e)

            # Record what the shifts are for each of the files run
            message='Reading in shift file: {file}'
            print(message.format(file=outshifts))
            shifts = Table.read(outshifts, format='ascii', names=('file',
                'xoffset','yoffset','rotation1','rotation2','scale1','scale2'))

            self.apply_tweakreg_success(shifts)

            # Add data from output shiftfile to shift_table
            for row in shifts:
                filename = os.path.basename(row['file'])
                filename = filename.replace('.rawtmp.fits','')
                filename = filename.replace('.fits','')

                idx = [i for i,row in enumerate(shift_table)
                    if filename in row['file']]

                if len(idx)==1:
                    shift_table[idx[0]]['xoffset']=row['xoffset']
                    shift_table[idx[0]]['yoffset']=row['yoffset']

            if not self.check_images_for_tweakreg(tmp_images):
                tweakreg_success = True

            tries += 1

    message = 'Tweakreg took {time} seconds to execute.\n\n'
    print(message.format(time = time.time()-start_tweak))

    print(shift_table)

    # tweakreg improperly indexes the CRVAL1 and CRVAL2 values
    # TODO: If drizzlepac fixes this then get rid of this code
    for image in tmp_images:
        rawtmp = image
        rawhdu = fits.open(rawtmp, mode='readonly')

        tweaksuc = False
        if ('TWEAKSUC' in rawhdu[0].header.keys() and
            rawhdu[0].header['TWEAKSUC']==1):
            tweaksuc = True

        if 'wfc3_ir' in self.get_instrument(image): continue

        for i,h in enumerate(rawhdu):
            if (tweaksuc and 'CRVAL1' in h.header.keys() and
                'CRVAL2' in h.header.keys()):
                rawhdu[i].header['CRPIX1']=rawhdu[i].header['CRPIX1']-0.5
                rawhdu[i].header['CRPIX2']=rawhdu[i].header['CRPIX2']-0.5

        rawhdu.writeto(rawtmp, overwrite=True)

    if not skip_wcs:
        for image in run_images:
            # Copy image over now to perform other image header updates
            if (image == reference or 'wfc3_ir' in self.get_instrument(image)):
                continue

            message = '\n\nUpdating image data for image: {im}'
            print(message.format(im=image))
            rawtmp = image.replace('.fits','.rawtmp.fits')

            rawhdu = fits.open(rawtmp, mode='readonly')
            hdu    = fits.open(image, mode='readonly')
            newhdu = fits.HDUList()

            print('Current image info:')
            hdu.info()

            for i, h in enumerate(hdu):
                if h.name=='SCI':
                    if 'flc' in image or 'flt' in image:
                        if len(rawhdu)>=i+2 and rawhdu[i+2].name=='DQ':
                            self.copy_wcs_keys(rawhdu[i], rawhdu[i+2])
                    elif 'c0m' in image:
                        maskfile = image.split('_')[0]+'_c1m.fits'
                        if os.path.exists(maskfile):
                            maskhdu = fits.open(maskfile)
                            self.copy_wcs_keys(rawhdu[i], maskhdu[i])
                            maskhdu.writeto(maskfile, overwrite=True)

                # Skip WCSCORR for WFPC2 as non-standard hdu
                if 'wfpc2' in self.get_instrument(image).lower():
                    if h.name=='WCSCORR':
                        continue

                # Get the index of the corresponding extension in rawhdu.  This
                # can be different from "i" if extensions were added or
                # rearranged
                ver = int(h.ver) ; name = str(h.name).strip()
                idx = -1

                for j,rawh in enumerate(rawhdu):
                    if str(rawh.name).strip()==name and int(rawh.ver)==ver:
                        idx = j

                # If there is no corresponding extension, then continue
                if idx < 0:
                    message = 'Skip extension {i},{ext},{ver} '
                    message += '- no match in {f}'
                    print(message.format(i=i, ext=name, ver=ver, f=rawtmp))
                    continue

                # If we can access the data in both extensions, copy from
                if h.name!='DQ':
                    if 'data' in dir(h) and 'data' in dir(rawhdu[idx]):
                        if (rawhdu[idx].data is not None and
                            h.data is not None):
                            if rawhdu[idx].data.dtype==h.data.dtype:
                                rawhdu[idx].data = h.data

                # Copy the rawtmp extension into the new file
                message = 'Copy extension {i},{ext},{ver}'
                print(message.format(i=idx, ext=name, ver=ver))
                newhdu.append(copy.copy(rawhdu[idx]))

            if 'wfpc2' in self.get_instrument(image).lower():
                # Adjust number of extensions to 4
                newhdu[0].header['NEXTEND']=4

            print('\n\nNew image info:')
            newhdu.info()

            newhdu.writeto(image, output_verify='silentfix', overwrite=True)

            if (os.path.isfile(rawtmp) and not self.options['args'].cleanup):
                os.remove(rawtmp)

    # Clean up temporary files and output
    if os.path.isfile('dummy.fits'):
        os.remove('dummy.fits')

    if not self.options['args'].keep_objfile:
        for file in glob.glob('*.coo'):
            os.remove(file)

    if modified:
        # Re-sanitize reference using main sanitize function
        self.sanitize_reference(reference)

    return(tweakreg_success, shift_table)